In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
import cvxpy as cp
from scipy import sparse

<font size="8">Pull data</font>

In [ ]:
def pull_data(Ticker_names, startDate, endDate, interval):
    """The pulling data function from Yahoo finance.

    Args:
        Ticker_names (list): List of ticker names in Yahoo finance that you want to pull.
        startDate (datetime): Starting date of data you want to pull in datetime form.
        endDate (datetime): Ending date of data you want to pull in datetime form.
        interval (str): Data interval. Valid intervals are:
                       “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”

    Returns:
        DataFrame: Close_price_dataframe
    """
    # Create dictionaries to store ticker objects and historical data
    Ticker = dict()
    Ticker_historical_data = dict()

    # Loop through ticker names to get ticker objects
    for ticker_name in Ticker_names:
        Ticker[ticker_name] = yf.Ticker(ticker_name)  # Get ticker object

    # Loop through ticker names to get historical data
    for ticker_name in Ticker_names:
        # Get historical data for each ticker
        Ticker_historical_data[ticker_name] = Ticker[ticker_name].history(
            start=startDate, end=endDate, interval=interval
        )[['Close']]
        # Rename the 'Close' column to the ticker name
        Ticker_historical_data[ticker_name].rename(columns={'Close': '%s' % ticker_name}, inplace=True)
        # Set the index to the date
        Ticker_historical_data[ticker_name].index = Ticker_historical_data[ticker_name].index.date

    # Concatenate the historical data for all tickers and drop rows with missing values
    return pd.concat(Ticker_historical_data.values(), axis=1).dropna()

def Log_return(Price_dataframe):
    """Compute the log return from an asset price dataframe.

    Args:
        Price_dataframe (DataFrame): Column: ticker name, Index: date

    Returns:
        DataFrame: Log_return_dataframe
    """
    # Create a shifted DataFrame by shifting Price_dataframe by one period
    dummy = Price_dataframe.shift(periods=1).T

    # Set the first column of the shifted DataFrame to the first row of Price_dataframe
    dummy[Price_dataframe.index[0]] = Price_dataframe.iloc[0].tolist()

    # Create a DataFrame for log returns by taking the natural logarithm of the ratio
    # of Price_dataframe to the shifted DataFrame (to calculate daily log returns)
    return pd.DataFrame(
        np.log(Price_dataframe.values.T / dummy.values),
        columns=Price_dataframe.index.tolist(),
        index=Price_dataframe.columns.tolist()
    ).T

def Mean_Cov(Log_return_dataframe, shifting_day):
    """Compute the Mean and Covariance of each asset.

    Args:
        Log_return_dataframe (DataFrame): Column: ticker name, Index: date
        shifting_day (int): The number of days to shift.

    Returns:
        (Mean, Cov): Mean list, Covariance matrix
    """
    # Compute the mean of log returns for each asset and multiply by the shifting days
    Mean = np.asarray(np.mean(Log_return_dataframe.values.T, axis=1)) * shifting_day

    # Compute the covariance matrix of log returns and multiply by the shifting days
    Cov = np.asmatrix(np.cov(Log_return_dataframe.values.T)) * shifting_day

    return Mean, Cov

def Mean_Variance_optimization(return_array, covariance_matrix, require_return, short_sell=None):
    """
    Optimize the portfolio for mean-variance using convex optimization.

    Args:
        return_array (array): Array of expected returns for each asset.
        covariance_matrix (array): Covariance matrix of asset returns.
        require_return (float): Required return for the portfolio.
        short_sell (str, optional): If short selling is allowed, input 'yes'. Defaults to None.

    Returns:
        tuple: Portfolio weights, objective value, and portfolio return.
    """
    # Define the decision variable for portfolio weights
    X = cp.Variable(len(return_array))

    # Formulate the objective function as the quadratic form of portfolio weights and covariance matrix
    objective_formulation = cp.quad_form(X, covariance_matrix)

    # Define the objective function to minimize the variance
    objective_function = cp.Minimize(objective_formulation)

    # Calculate the expected return of the portfolio
    Return = return_array.T @ X

    # Define constraints based on the specified conditions (short selling or not)
    if short_sell == None:

        constraints = [sum(X) == 1, Return >= require_return, X >= 0]
    else:

        constraints = [sum(X) == 1, Return >= require_return]

    constraints = constraints + [X <= np.array([0.2 for i in range(len(return_array))])]

    # Formulate the convex optimization problem
    problem = cp.Problem(objective_function, constraints)

    # Solve the optimization problem
    problem.solve()

    if problem.status != 'optimal':

        optimal_portfolio = np.array([0 for i in range(len(return_array))])

    else:

        optimal_portfolio = X.value

    # Return the optimized portfolio weights, objective value (variance), and the expected return of the portfolio
    return optimal_portfolio, problem.value, Return.value

def checking_day_trade(Date, Close_price_dataframe):
    """
    Check if the given date is a trading day. If not, find the next trading day.

    Args:
        Date (datetime): The input date to check.
        Close_price_dataframe (DataFrame): DataFrame with trading day indices.

    Returns:
        datetime: The next trading day date.
    """
    # Continue searching for the next trading day until a trading day is found
    while not (Date.date() in Close_price_dataframe.index):
        # Increment the date by one day
        Date = Date + timedelta(days=1)

    # Return the next trading day date
    return Date

def Backtesting_Mean_Variance(len_of_trianing_data, trading_day, period, number_of_trading, require_return, Close_price_dataframe, short_sell = None):
    """
    Backtest a Mean-Variance portfolio strategy over multiple trading periods.

    Args:
        len_of_trianing_data (int): Length of the training data period (in months).
        trading_day (datetime): Initial trading day to start the backtesting.
        period (int): Number of months for each trading period.
        number_of_trading (int): Number of trading periods to backtest.
        require_return (float): Required return for the portfolio in each trading period.
        Close_price_dataframe (DataFrame): DataFrame with close prices for each trading day.

    Returns:
        dict, dict: Returns and portfolio proportions for each trading period.
    """
    trading_day = checking_day_trade(trading_day, Close_price_dataframe)
    start_dict = dict()
    end_dict = dict()
    trianing_data = dict()
    day_trading = dict()
    Mean_dict = dict()
    Cov_dict = dict()
    portfolio_proportions_dict = dict()

    # Loop through each trading period for backtesting
    for i in range(number_of_trading + 1):
        start_dict[i] = (trading_day - len_of_trianing_data * timedelta(days=30)).date()
        end_dict[i] = (trading_day - timedelta(days=1)).date()
        trianing_data[i] = Close_price_dataframe.loc[start_dict[i]:end_dict[i]]
        day_trading[i] = trading_day
        Mean_dict[i], Cov_dict[i] = Mean_Cov(Log_return(trianing_data[i]), period * 30)
        portfolio_proportions_dict[i], __, __ = Mean_Variance_optimization(Mean_dict[i], Cov_dict[i], require_return, short_sell=short_sell)

        trading_day = checking_day_trade(trading_day + timedelta(days=30 * period), Close_price_dataframe)

    R = dict()
    # Calculate returns for each trading period
    for i in range(number_of_trading):
        R[i + 1] = sum(portfolio_proportions_dict[i] * np.log(np.array(Close_price_dataframe.loc[day_trading[i + 1].date()].tolist()) /
                                                       np.array(Close_price_dataframe.loc[day_trading[i].date()].tolist())))

    return R, portfolio_proportions_dict, day_trading





In [ ]:
Ticker_names = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'META', 'TSLA', 'NVDA', 'JNJ', 'JPM', 'PG', 'META', 'V', 'UNH', 'MA', 'HD', 'INTC', 'VZ', 'CSCO', 'T', 'BAC', 'DIS', 'ADBE', 'CMCSA','MRK']
startDate = datetime.datetime(2018, 1, 2)
endDate = datetime.datetime(2021, 12, 31)
interval = '1d'

# Call the pull_data function to retrieve historical close prices for the specified tickers and date range
Close_price_dataframe = pull_data(Ticker_names, startDate, endDate, interval)

# Display the resulting dataframe containing historical close prices
Close_price_dataframe

<ipython-input-32-5cad58badda3>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,AAPL,MSFT,AMZN,GOOGL,GOOG,META,TSLA,NVDA,JNJ,JPM,...,HD,INTC,VZ,CSCO,T,BAC,DIS,ADBE,CMCSA,MRK
2018-01-02,40.722878,80.229004,59.450500,53.660500,53.250000,181.419998,21.368668,49.312782,118.394844,90.681252,...,162.567657,39.857128,39.690174,32.407032,18.058531,25.995726,108.372864,177.699997,35.979404,44.663513
2018-01-03,40.715790,80.602386,60.209999,54.576000,54.124001,184.669998,21.150000,52.558254,119.525826,90.773682,...,163.414932,38.504452,38.874580,32.665558,17.641508,25.908789,108.838142,181.039993,35.401203,44.599957
2018-01-04,40.904907,81.311813,60.479500,54.787998,54.320000,184.330002,20.974667,52.835316,119.517342,92.074051,...,164.711823,37.798344,39.000629,32.757969,17.805502,26.247864,108.789680,183.220001,35.628983,45.322899
2018-01-05,41.370621,82.319901,61.457001,55.514500,55.111500,186.850006,21.105333,53.283031,120.503754,91.482948,...,166.432343,38.062073,38.911648,33.211655,17.871101,26.369587,108.198380,185.339996,35.953125,45.275223
2018-01-08,41.216965,82.403915,62.343498,55.710499,55.347000,188.279999,22.427334,54.915668,120.656792,91.618065,...,166.034622,38.062073,38.844921,33.556126,17.922647,26.187004,106.647423,185.039993,35.462532,45.013065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,174.288635,328.668732,171.068497,146.916504,147.142502,335.239990,355.666656,295.991760,159.114304,148.035553,...,376.839172,48.225643,46.528503,58.588699,16.380890,42.098457,153.130936,569.619995,47.347988,71.374336
2021-12-27,178.292877,336.289154,169.669495,147.906494,148.063995,346.179993,364.646667,309.023804,160.457184,148.882767,...,383.501465,48.817772,46.528503,59.661133,16.321609,42.306961,152.303635,577.679993,47.764400,72.166023
2021-12-28,177.264633,335.110687,170.660995,146.686996,146.447998,346.220001,362.823334,302.802368,161.100281,149.334625,...,385.522980,48.648594,46.563835,59.764618,16.347956,42.363831,154.695831,569.359985,47.802261,72.392212
2021-12-29,177.353607,335.798157,169.201004,146.654999,146.504501,342.940002,362.063324,299.596802,162.235107,149.259293,...,389.907532,48.714390,46.245876,60.169125,16.229399,42.297489,154.366898,569.289978,47.877968,72.524170


In [ ]:
Log_return_datafrme = Log_return(Close_price_dataframe)
Log_return_datafrme

,AAPL,MSFT,AMZN,GOOGL,GOOG,META,TSLA,NVDA,JNJ,JPM,...,HD,INTC,VZ,CSCO,T,BAC,DIS,ADBE,CMCSA,MRK
2018-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-01-03,-0.000174,0.004643,0.012694,0.016917,0.016280,0.017756,-0.010286,0.063739,0.009507,0.001019,...,0.005198,-0.034527,-0.020763,0.007946,-0.023364,-0.003350,0.004284,0.018621,-0.016201,-0.001424
2018-01-04,0.004634,0.008763,0.004466,0.003877,0.003615,-0.001843,-0.008325,0.005258,-0.000071,0.014224,...,0.007905,-0.018509,0.003237,0.002825,0.009253,0.013002,-0.000445,0.011970,0.006414,0.016080
2018-01-05,0.011321,0.012322,0.016033,0.013173,0.014466,0.013579,0.006210,0.008438,0.008219,-0.006441,...,0.010391,0.006953,-0.002284,0.013755,0.003677,0.004627,-0.005450,0.011504,0.009057,-0.001052
2018-01-08,-0.003721,0.001020,0.014322,0.003524,0.004264,0.007624,0.060755,0.030181,0.001269,0.001476,...,-0.002393,0.000000,-0.001716,0.010319,0.002880,-0.006948,-0.014438,-0.001620,-0.013739,-0.005807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,0.003637,0.004462,0.000184,0.003419,0.001316,0.014391,0.056020,0.008130,0.001904,0.003567,...,0.003608,0.006649,-0.001707,0.012115,0.003625,0.003157,0.011456,0.009951,0.020190,-0.005662
2021-12-27,0.022715,0.022921,-0.008212,0.006716,0.006243,0.032112,0.024935,0.043087,0.008404,0.005707,...,0.017525,0.012204,0.000000,0.018139,-0.003625,0.004941,-0.005417,0.014051,0.008756,0.011031
2021-12-28,-0.005784,-0.003510,0.005827,-0.008279,-0.010974,0.000116,-0.005013,-0.020338,0.004000,0.003030,...,0.005257,-0.003472,0.000759,0.001733,0.001613,0.001343,0.015585,-0.014507,0.000792,0.003129
2021-12-29,0.000502,0.002049,-0.008592,-0.000218,0.000386,-0.009519,-0.002097,-0.010643,0.007020,-0.000505,...,0.011309,0.001352,-0.006852,0.006746,-0.007279,-0.001567,-0.002129,-0.000123,0.001582,0.001821


In [ ]:
shifting_day = 360

Mean, Cov = Mean_Cov(Log_return_datafrme, shifting_day)
print('Mean array is',Mean)
print('Covariance matrix is',Cov)

Mean array is [ 0.52364647  0.50904218  0.3727444   0.35832041  0.36058128  0.22911044
  1.00642473  0.6400366   0.11419969  0.1779729   0.24915606  0.23899921
  0.31638303  0.31702631  0.31196033  0.07111888  0.05389829  0.21930825
 -0.03615309  0.17322627  0.12890488  0.4170098   0.10214017  0.1741831 ]
Covariance matrix is [[0.15478407 0.10666588 0.09492916 0.09281581 0.09269426 0.10277566
  0.13004562 0.14620426 0.04412008 0.07047156 0.04365402 0.08478935
  0.07345112 0.09662685 0.07789895 0.10030289 0.02517317 0.08251813
  0.04277722 0.07813425 0.0640579  0.11343398 0.0602424  0.04286412]
 [0.10666588 0.12732325 0.0948843  0.09946287 0.09852699 0.09884333
  0.1228854  0.14528214 0.04696122 0.07043436 0.04571227 0.08948383
  0.07714244 0.09984611 0.08025255 0.10185881 0.03025317 0.08415687
  0.04205959 0.07726295 0.0658044  0.12470192 0.0636732  0.0485366 ]
 [0.09492916 0.0948843  0.13879565 0.08707602 0.08726987 0.09915157
  0.11128832 0.13512939 0.02935996 0.04117338 0.02790724 0

In [ ]:
require_return_2 = 0.12
optimal_Portfolio2,Risk,__ = Mean_Variance_optimization(Mean, Cov, require_return_2)

In [ ]:
print(dict(zip(Ticker_names,optimal_Portfolio2)))

{'AAPL': -2.725025192548916e-23, 'MSFT': 2.0224655291980476e-24, 'AMZN': 0.14354465214066148, 'GOOGL': 1.5417878086970522e-25, 'GOOG': 1.1753743676906459e-23, 'META': 5.699012190818893e-24, 'TSLA': 1.379739348609081e-23, 'NVDA': -1.9941218561214415e-23, 'JNJ': 0.18618823587481775, 'JPM': 6.467291078475759e-25, 'PG': 0.2, 'V': -8.353210445528897e-24, 'UNH': -3.209673589744005e-23, 'MA': -2.0493316427403766e-23, 'HD': -1.6198723171133155e-24, 'INTC': 0.2, 'VZ': -1.0370756432914151e-23, 'CSCO': 0.06857062488337348, 'T': -8.242601623817178e-24, 'BAC': 0.0411705697404212, 'DIS': 2.974315882062815e-23, 'ADBE': 1.0566230869979832e-23, 'CMCSA': 0.16052591736072602}


In [ ]:
hist = go.Figure(go.Bar(x= Ticker_names,
    y=optimal_Portfolio2,text=np.round(optimal_Portfolio2,decimals=2),textposition='auto'))

hist.update_layout(xaxis_title = 'Asset', yaxis_title = 'Proportion', title = 'Portfolio Proportion')
hist.show()